# <font color="#418FDE" size="6.5" uppercase>**Raising And Handling**</font>

>Last update: 20251221.
    
By the end of this Lecture, you will be able to:
- Raise built-in exceptions with informative messages that aid debugging and user feedback. 
- Use raise from to chain exceptions and preserve original error context when wrapping lower-level failures. 
- Format and log exceptions using built-ins like str, repr, and print for clear diagnostics. 


## **1. Raising Exceptions Clearly**

### **1.1. Raising Exception Instances**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_01_01.jpg?v=1766326026" width="250">



>* Raise exceptions instead of continuing with bad state
>* Use specific built-in types to signal exact problems

>* Exception instances should explain what went wrong
>* Include specific context to guide debugging and users

>* Consistent, specific exceptions improve communication and maintainability
>* Clear exception patterns enable robust handling and diagnosis



In [ ]:
#@title Python Code - Raising Exception Instances

# Demonstrate raising exception instances with clear informative messages.
# Show how invalid input triggers deliberate exception signaling behavior.
# Help beginners see why raising exceptions beats silent incorrect processing.

from typing import Any, Tuple, List


def calculate_paint_cans(room_length_feet: float, room_width_feet: float) -> int:
    if room_length_feet <= 0 or room_width_feet <= 0:
        raise ValueError("Room dimensions must be positive feet, received nonpositive value.")
    room_area_square_feet: float = room_length_feet * room_width_feet
    coverage_per_can_square_feet: float = 350.0
    cans_needed_exact: float = room_area_square_feet / coverage_per_can_square_feet
    cans_needed_rounded: int = int(-(-cans_needed_exact // 1))
    return cans_needed_rounded


example_inputs: List[Tuple[Any, Any]] = [(12.0, 10.0), (15.0, -8.0), (0.0, 9.0)]


for length_feet, width_feet in example_inputs:
    print("Trying room size feet:", length_feet, "by", width_feet)
    try:
        cans: int = calculate_paint_cans(length_feet, width_feet)
        print("Paint cans required:", cans)
    except ValueError as error:
        print("Raised ValueError instance message:", str(error))





### **1.2. Exception Messages and Data**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_01_02.jpg?v=1766326070" width="250">



>* Clear exception messages explain what failed and why
>* Good wording guides fixes and documents code assumptions

>* Include concrete values and locations in errors
>* Add identifiers and constraints for easier debugging

>* Hide sensitive details; use safe identifiers instead
>* Log technical detail, show simpler messages to users



In [ ]:
#@title Python Code - Exception Messages and Data

# Demonstrate clear exception messages with helpful contextual data.
# Compare vague and detailed messages for the same validation failure.
# Show safe logging that avoids exposing sensitive user information.

from dataclasses import dataclass

@dataclass
class BookingRequest:
    check_in_date: str
    check_out_date: str


def validate_booking(request: BookingRequest) -> None:
    if request.check_in_date >= request.check_out_date:
        message = (
            "Check-in date must be before check-out date; "
            f"received check_in={request.check_in_date}, "
            f"check_out={request.check_out_date}."
        )
        raise ValueError(message)


def process_booking(request: BookingRequest) -> None:
    try:
        validate_booking(request)
    except ValueError as error:
        safe_reference = "REF-4829"
        technical_message = f"Booking failed {safe_reference}: {error}"
        user_message = (
            "We could not complete your booking; "
            f"please contact support with reference {safe_reference}."
        )
        print("Technical log message:")
        print(technical_message)
        print("User facing message:")
        print(user_message)


bad_request = BookingRequest(check_in_date="2025-12-10", check_out_date="2025-12-05")
process_booking(bad_request)



### **1.3. Reraising Exceptions Safely**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_01_03.jpg?v=1766326118" width="250">



>* Catch exceptions to log, clean up, add context
>* Reraise so higher layers decide final handling

>* Avoid replacing specific errors with vague ones
>* Log full technical details while showing friendly messages

>* Lower layers rethrow errors, adding helpful context
>* Top layers convert errors into clear user messages



In [ ]:
#@title Python Code - Reraising Exceptions Safely

# Demonstrate safe exception reraising with added context information.
# Show logging behavior before reraising original exceptions upward.
# Compare bad reraising with good reraising using simple functions.

import random


def low_level_division(miles_distance, gallons_fuel):
    """Divide distance by fuel and maybe raise ZeroDivisionError."""
    print("Low level dividing miles by gallons now.")
    return miles_distance / gallons_fuel


def mid_level_trip_calculation(miles_distance, gallons_fuel):
    """Catch, log, and safely reraise original division exception."""
    try:
        mpg_value = low_level_division(miles_distance, gallons_fuel)
    except ZeroDivisionError as error:
        print("Mid level logged division failure details:", error)
        raise
    else:
        return mpg_value


def bad_reraising_example(miles_distance, gallons_fuel):
    """Replace original error with vague message, losing helpful context."""
    try:
        return low_level_division(miles_distance, gallons_fuel)
    except ZeroDivisionError:
        raise RuntimeError("Trip calculation failed for unknown reason.")


def run_demo():
    """Run both examples and show different exception messages clearly."""
    print("Running safe reraising example with zero gallons now.")
    try:
        mid_level_trip_calculation(120, 0)
    except Exception as error:
        print("Caller received exception type and message:", type(error).__name__, str(error))

    print()

    print("Running bad reraising example with zero gallons now.")
    try:
        bad_reraising_example(120, 0)
    except Exception as error:
        print("Caller only sees vague replacement error:", type(error).__name__, str(error))


run_demo()



## **2. Exception Chaining Basics**

### **2.1. Automatic Exception Chaining**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_02_01.jpg?v=1766326165" width="250">



>* Runtime links new exceptions to current ones
>* Final traceback shows sequence of related failures

>* Second error can hide the real cause
>* Automatic chaining keeps both immediate and original errors

>* Chaining links high-level and low-level errors
>* Developers trace error chains to real root cause



In [ ]:
#@title Python Code - Automatic Exception Chaining

# Demonstrate automatic exception chaining during nested error handling.
# Show how a second exception remembers the original failure context.
# Print the final traceback and observe the chained exceptions.

import traceback as tb_module


def read_config_value():
    raise FileNotFoundError("Main configuration file missing in current directory.")


def handle_config_and_log():
    try:
        read_config_value()
    except FileNotFoundError as original_error:
        print("Handling configuration error, attempting fallback logging.")
        raise RuntimeError("Logging system failed while handling configuration error.")


try:
    handle_config_and_log()
except Exception as final_error:
    print("\nFinal exception traceback with automatic chaining shown below.")
    tb_module.print_exception(type(final_error), final_error, final_error.__traceback__)



### **2.2. Using raise from explicitly**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_02_02.jpg?v=1766326215" width="250">



>* Use raise from to relate layered errors
>* Wrap low-level failures while keeping original cause

>* Use raise from to hide low-level details
>* Keep original error context available for developers

>* Use raise from to separate layer-specific details
>* Keep both high-level messages and original causes



In [ ]:
#@title Python Code - Using raise from explicitly

# Demonstrate explicit exception chaining using raise from syntax.
# Show low level failure wrapped by higher level domain exception.
# Print both exceptions to observe preserved original error context.

class PaymentGatewayError(Exception):
    """Custom payment gateway error representing higher level failure."""


def low_level_network_call(card_number: str) -> str:
    """Simulate low level network call that may raise ValueError."""
    if not card_number.isdigit():
        raise ValueError("Card number contains non digit characters.")
    if len(card_number) != 16:
        raise ValueError("Card number must contain sixteen digits exactly.")
    return "OK"


def process_payment(card_number: str) -> None:
    """Wrap low level error using raise from for clearer domain message."""
    try:
        result = low_level_network_call(card_number)
        print("Low level network call succeeded with result:", result)
    except ValueError as original_error:
        message = "Payment processing failed due to invalid card details provided."
        raise PaymentGatewayError(message) from original_error


try:
    print("Attempting payment with clearly invalid card number now.")
    process_payment("1234-INVALID-CARD")
except PaymentGatewayError as high_error:
    print("Caught PaymentGatewayError with message:", high_error)
    print("Original cause type and message:", type(high_error.__cause__), high_error.__cause__)



### **2.3. Inspecting Exception Links**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_02_03.jpg?v=1766326261" width="250">



>* Inspect chained exceptions to reveal hidden root causes
>* Trace errors across layers for deeper debugging insight

>* Inspect chains via tracebacks and exception attributes
>* Traceback order reveals error flow across layers

>* Programmatically walk exception chains for richer logging
>* Use root causes to drive alerts and recovery



In [ ]:
#@title Python Code - Inspecting Exception Links

# Demonstrate chained exceptions and their links clearly.
# Show how __cause__ and __context__ attributes connect exceptions.
# Print traceback and walk the chain programmatically for inspection.

import traceback as tb_module


def low_level_division(miles_value, hours_value):
    # Perform fragile division that may raise ZeroDivisionError directly.
    return miles_value / hours_value


def mid_level_speed_calculation():
    # Call low level function and wrap any error using raise from syntax.
    try:
        return low_level_division(60, 0)
    except ZeroDivisionError as original_error:
        raise ValueError("Speed calculation failed due division.") from original_error


def show_traceback_and_links():
    # Call mid level function and catch the chained exception for inspection.
    try:
        mid_level_speed_calculation()
    except Exception as caught_error:
        print("Printed traceback with chained exceptions:")
        tb_module.print_exception(type(caught_error), caught_error, caught_error.__traceback__)

        print("\nWalking exception chain using __cause__ attribute:")
        current_error = caught_error
        while current_error is not None:
            print(f"Type: {type(current_error).__name__}, Message: {current_error}")
            current_error = current_error.__cause__


show_traceback_and_links()



## **3. Displaying Exceptions Clearly**

### **3.1. Readable Exception Messages**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_03_01.jpg?v=1766326305" width="250">



>* Exception messages should explain what failed and why
>* Specific, detailed messages make diagnosing errors much easier

>* Match error detail to audience and context
>* Use clear, specific messages that guide action

>* Include specific context like operation, resource, value
>* Be informative but avoid exposing sensitive data



In [ ]:
#@title Python Code - Readable Exception Messages

# Show how readable exception messages improve debugging clarity.
# Compare vague and descriptive error messages for the same failure.
# Demonstrate printing exceptions using str for user friendly output.

# This function simulates loading a configuration value from user input.
# It raises ValueError with either vague or descriptive messages.
# The descriptive message explains what failed and how to fix it.

def parse_max_connections(raw_value, descriptive_message_enabled):
    if not raw_value.isdigit():
        if descriptive_message_enabled:
            raise ValueError(
                "Invalid value '" + raw_value + "' for setting 'max_connections'; expected integer."
            )
        else:
            raise ValueError("Invalid configuration error occurred.")
    return int(raw_value)

# Helper function runs the parser and prints the resulting exception message.
# It uses str(exception) to show the human readable message text.

def demo_run(label_text, raw_value, descriptive_message_enabled):
    print("Scenario:", label_text)
    try:
        result_value = parse_max_connections(raw_value, descriptive_message_enabled)
        print("Parsed value:", result_value)
    except ValueError as error_instance:
        print("Caught error message:", str(error_instance))

# Run two scenarios with the same bad input but different messages.
# The second scenario shows a clearer, more helpful explanation.

bad_input_value = "thirty"

demo_run("Vague message example", bad_input_value, False)

print()

demo_run("Descriptive message example", bad_input_value, True)



### **3.2. Debugging With repr**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_03_02.jpg?v=1766326358" width="250">



>* Use repr to see true data structure
>* Repr exposes hidden characters and reveals root causes

>* repr exposes full structure of complex data
>* Helps spot subtle issues hidden by friendly strings

>* repr stays consistent across systems and logs
>* Helps reconstruct exact program state for debugging



In [ ]:
#@title Python Code - Debugging With repr

# Show difference between str and repr for debugging exceptions.
# Demonstrate hidden whitespace and types inside error messages clearly.
# Use simple failing conversion and dictionary lookup examples for clarity.

bad_distance_text = "  42 \n"
print("User friendly str shows:", str(bad_distance_text))
print("Developer focused repr shows:", repr(bad_distance_text))
print("Notice spaces and newline revealed by repr clearly.")

try:
    miles = int(bad_distance_text)
    print("Converted miles value is:", miles)
except ValueError as error:
    print("ValueError str message is:", str(error))
    print("ValueError repr message is:", repr(error))

payload = {"amount": "19.99", "currency": "usd"}
print("Payload str view shows:", payload)
print("Payload repr view shows:", repr(payload))
missing_value = payload.get("tax_rate", None)
print("Missing tax_rate repr shows:", repr(missing_value))



### **3.3. Printing Stack Traces**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_08/Lecture_B/image_03_03.jpg?v=1766326401" width="250">



>* Stack traces show the call path to errors
>* They pinpoint where failures start for easier debugging

>* Stack traces reveal the full call path
>* They pinpoint root cause and responsible module

>* Log stack traces to capture production errors
>* Use stored traces to spot patterns and improve



In [ ]:
#@title Python Code - Printing Stack Traces

# Demonstrate how Python prints stack traces for debugging clarity.
# Show nested function calls that raise and display an exception.
# Use traceback module to print a readable stack trace example.

import traceback  # Import traceback module for printing stack traces.


def inches_to_miles(inches_value):  # Convert inches to miles with validation.
    if inches_value < 0:  # Negative inches are invalid and should raise error.
        raise ValueError("Inches value cannot be negative.")
    return inches_value / 63360  # Convert inches to miles using constant.


def trip_distance_in_miles(speed_mph, hours_driven):  # Compute trip distance.
    inches_per_mile = 63360  # Define inches per mile conversion constant.
    total_inches = speed_mph * hours_driven * inches_per_mile
    return inches_to_miles(total_inches)  # Delegate conversion to helper.


print("Starting trip distance calculation now.")  # Announce calculation start.

try:
    result = trip_distance_in_miles(-55, 2)  # Intentionally use negative speed.
    print("Trip distance in miles:", result)
except Exception as error:  # Catch any raised exception for demonstration.
    print("An error occurred during calculation.")
    print("Exception message only:", str(error))
    print("\nFull stack trace for debugging:")
    traceback.print_exc()  # Print full stack trace to understand failure.




# <font color="#418FDE" size="6.5" uppercase>**Raising And Handling**</font>


In this lecture, you learned to:
- Raise built-in exceptions with informative messages that aid debugging and user feedback. 
- Use raise from to chain exceptions and preserve original error context when wrapping lower-level failures. 
- Format and log exceptions using built-ins like str, repr, and print for clear diagnostics. 

In the next Module (Module 9), we will go over 'I O And Environment'